# Import resources into eLab database


This script will read a csv file that contains a list of antibodies, and add them to the resources database with a fine control on which columns are processed and how.

## Tech detail

In [1]:
# the python lib for elab
import elabapi_python
# we will use the generic csv module
import csv
# we also need the json module for the metadata (extra fields) part
import json

from pathlib import Path
import os
from dotenv import load_dotenv


import pandas as pd # to create a combined dataframe

In [28]:
#########################
#         CONFIG        #
#########################
# replace with your instance address
load_dotenv()
API_HOST_URL = os.getenv('API_HOST_URL') #a secure way to store and load sensitive info from a hidden .env file


#Available endpoints (not empty) are: 
#     apikeys, 
#     config, 
#     experiments, 
#     info, 
#     items, #this is resources database
#     experiments_templates, 
#     items_types, 
#     event, 
#     events, 
#     extra_fields_keys,
#     team_tags, 
#     teams, 
#     todolist, 
#     unfinished_steps, 
#     users

# replace with your api key
API_KEY = os.getenv('eLabFTW_API_KEY') #a secure way to store and load sensitive info incl. API keys from a hidden .env file

# this is the resource category where the entries will be created. Visit https://eln.ibecbarcelona.eu/api/v2/items_types to GET a list.

# in this example, category with id 3 corresponds to "Project CRYPTO-COOL" at IBEC
# in this example, category with id 6 corresponds to "Antibodies" at DEMO
# RESOURCE_CATEGORY_ID = 15 #IBEC Antibodies (Primary)
# RESOURCE_CATEGORY_ID = 59 #IBEC Antibodies (Secondary)
# RESOURCE_CATEGORY_ID = 60 #IBEC Fluorophores GBQ
# RESOURCE_CATEGORY_ID = 61 #IBEC General Supply
# RESOURCE_CATEGORY_ID = 62 #IBEC Reagents
# RESOURCE_CATEGORY_ID = 63 #IBEC Equipment
# RESOURCE_CATEGORY_ID = 68 #IBEC Solvents


#RESOURCE_CATEGORY_ID = 6 #DEMO

# parent_dir is a parent directory to our current directory `cwd`
parent_dir = Path.cwd().parent

# relative_path is a relative path to the directory where our data is stored
relative_path = 'Data/'

#########################
#      END CONFIG       #
#########################

In [34]:
# Configure the api client
configuration = elabapi_python.Configuration()
configuration.api_key['api_key'] = API_KEY
configuration.api_key_prefix['api_key'] = 'Authorization'
configuration.host = API_HOST_URL
configuration.debug = False
# set to True if you have a proper certificate, here it is set to False to ease the test in dev
configuration.verify_ssl = False

# create an instance of the API class
api_client = elabapi_python.ApiClient(configuration)
# fix issue with Authorization header not being properly set by the generated lib
api_client.set_default_header(header_name='Authorization', header_value=API_KEY)
    
# Load items api
itemsApi = elabapi_python.ItemsApi(api_client)

# Load the experiments api
experimentsApi = elabapi_python.ExperimentsApi(api_client)


## Helper functions to import resources from an csv file

### For the example data

In [16]:
# function to build the metadata json for a row
def getMetadataFromRowExample(row):
    # our metadata object for one row, currently a dictionary with a key "extra_fields" holding an empty dictionary
    metadata = { 'extra_fields': {} }

    # now go over the columns (except the title/Name) and add it to our extra_fields object
    for keyval in row.items():
        field_type = 'text'

        # we don't import these columns as metadata
        # Name is the title, Comment is in the body, and ID is the custom_id.
        if keyval[0] == 'Name' or keyval[0] == 'Comment' or keyval[0] == 'ID':
            continue
        
        # special case for url/URL column, we make it a type: url
        if keyval[0].lower() == 'url':
            field_type = 'url'
        
        if keyval[0].lower() == 'price':
            field_type = 'number'
        
        # special case for Concentration column, we use the units
        if keyval[0].lower() == 'concentration' and keyval[1]:
            split_conc = keyval[1].split()
            metadata['extra_fields'].update({keyval[0]: {'value': split_conc[0], 'type': 'number', 'unit': split_conc[1], 'units':['mg/mL', 'μg/mL']}})
        elif keyval[0].lower() == 'primary vs secondary':
            metadata['extra_fields'].update({keyval[0]: {'value': 'Primary', 'type': 'select', 'options': ['Primary', 'Secondary']}})

        elif keyval[0].lower() == 'raised in':
            metadata['extra_fields'].update({keyval[0]: {'value': keyval[1], 'type': 'select', 'options': ['Rabbit', 'Mouse']}})
        elif keyval[0].lower() == 'recognizes':
            metadata['extra_fields'].update({keyval[0]: {
                'value': keyval[1].split(', '), 'type': 'select', 'allow_multi_values': True, 'options': ['Ape', 'Chicken', 'Dog', 'Goat', 'Guinea Pig', 'Hamster', 'Human', 'Mink', 'Monkey', 'Mouse', 'Rabbit', 'Rat', 'Sheep', 'Zebrafish']}})
        else:
            metadata['extra_fields'].update({keyval[0]: {'value': keyval[1], 'type': field_type}})
    
    return json.dumps(metadata)

In [17]:
# The column "Comment" will get added to the body of the resource
def getBodyFromRowExample(row) -> str:
    for keyval in row.items():
        if keyval[0] == 'Comment':
            return f'<p>{keyval[1]}</p>'
    return ''


### For the MB test data we combine all the excel sheets in one csv file, and upload it to the eLabFTW system

In [12]:
import datetime
# Specifying our Quartzy Excel file path

filename_MB_to_merge = 'Molecular_Bionics_Inventory_Jun_12_2024_to_merge.xlsx'

EXCEL_PATH = (parent_dir / relative_path / filename_MB_to_merge).resolve()

# Specifying the sheet names to extract
sheet_names = ['Antibodies (primary)', 
               'Antibodies (secondary)', 
               'Equipment', 
               'Fluorophores GBQ', 
               'General Supply', 
               'Reagents',
               'Solvents']

# Reading the Excel file and extract the specified sheets
# Creating a dictionary of dataframes, where the key is the sheet name and the value is the dataframe
dfs = pd.read_excel(EXCEL_PATH, sheet_name=sheet_names) 

# Adding the 'Type' column to each dataframe
for sheet_name, df in dfs.items():
    df['Type'] = sheet_name

# Combining the dataframes into one
combined_df = pd.concat(dfs.values())

# Merging 'Molecular weight' and 'Molecular Weight *' into 'Molecular weight'
combined_df['Molecular Weight'] = combined_df['Molecular Weight'].fillna(combined_df['Molecular Weight *'])

# Removing 'Molecular Weight *' column
combined_df = combined_df.drop(columns=['Molecular Weight *'])

# Moving the 'Type' column to the first position
cols = combined_df.columns.tolist()
cols = ['Type'] + [col for col in cols if col != 'Type']
combined_df = combined_df[cols]

# Adding prefix to 'Name *' column
combined_df['Item Name *'] = '(Created via API) ' + combined_df['Item Name *']

# Get the current date
current_date = datetime.datetime.now().strftime("%Y-%m-%d")

# Adding date to the filename
filename_with_date = f"Molecular_Bionics_Inventory_Merged_{current_date}.csv"

# Saving the combined dataframe to the dated CSV file
combined_df.to_csv(filename_with_date, index=False)


In [37]:
# function to build the metadata json for a row
def getMetadataFromRow(row):
    # our metadata object for one row, currently a dictionary with a key "extra_fields" holding an empty dictionary
    metadata = { 'extra_fields': {} }

    # now go over the columns (except the title/Name) and add it to our extra_fields object
    for keyval in row.items():
        field_type = 'text'

        # we don't import these columns as metadata
        # 'Item Name' is the title, 'Technical Details' is body, and 'Serial Number' is the custom_id.
        if keyval[0] == 'Item Name *' or keyval[0] == 'Technical Details' or keyval[0] == 'Serial Number':
            continue
        
        # special case for url/URL column, we make it a type: url
        if keyval[0].lower() == 'URL':
            field_type = 'url'
        
        if keyval[0].lower() == 'Price' or keyval[0].lower() == 'Molecular Weight'  or keyval[0].lower() == 'Min to Stock' or keyval[0].lower() == 'Max to Stock':
            field_type = 'number'

        if keyval[0].lower() == 'Expiration Date' or keyval[0].lower() == 'Date Installed' or keyval[0].lower() == 'Date Opened' or keyval[0].lower() == 'Date Purchased':
            field_type = 'date'
        
        # special case for Concentration column, we use the units
        if keyval[0].lower() == 'Concentration' and keyval[1]:
            split_conc = keyval[1].split()
            metadata['extra_fields'].update({keyval[0]: {'value': split_conc[0], 'type': 'number', 'unit': split_conc[1], 'units':['mg/mL', 'μg/mL']}})

        elif keyval[0].lower() == 'Primary vs Secondary':
            metadata['extra_fields'].update({keyval[0]: {'value': 'Primary', 'type': 'select', 'options': ['Primary', 'Secondary']}})

        # this is from eLabFTW example and fits Quartzy
        elif keyval[0].lower() == 'Raised in':
            metadata['extra_fields'].update({keyval[0]: {'value': keyval[1], 'type': 'select', 'options': ['Chicken', 'Donkey', 'Goat', 'Hamster', 'Human', 'Mouse', 'Rabbit', 'Rat']}})
        
        # this is from eLabFTW example and fits Quartzy
        elif keyval[0].lower() == 'Recognizes':
            metadata['extra_fields'].update({keyval[0]: {
                'value': keyval[1].split(', '), 'type': 'select', 'allow_multi_values': True, 'options': ['Ape', 'Bacteria', 'Bovine', 'Budding Yeast', 'Canine', 'Chicken', 'Feline', 'Fission Yeast', 'Fruit Fly', 'Frog', 'Goat','Guinea Pig','Hamster', 'Human', 'Mink', 'Monkey', 'Mouse', 'Mustard Weed', 'Nematode', 'Protein Tag', 'Rabbit', 'Rat', 'Sea Slug', 'Sheep', 'Zebrafish']}})
            
        # this is from Quartzy
        elif keyval[0].lower() == 'Applications':
            metadata['extra_fields'].update({keyval[0]: {
            'value': keyval[1].split(', '), 'type': 'select', 'allow_multi_values': True, 'options': ['Affinity Purification', 'ChIP', 'ELISA', 'Flow cytometry', 'Immunofluorescence', 'Immunohistochemistry', 'Immunoprecipitation', 'Western Blot']}})

        # this is from Quartzy
        elif keyval[0].lower() == 'Clonality':
            metadata['extra_fields'].update({keyval[0]: {
            'value': keyval[1], 'type': 'select', 'options': ['Monoclonal', 'Polyclonal']}})

        # this is from Quartzy
        elif keyval[0].lower() == 'User':
            metadata['extra_fields'].update({keyval[0]: {
            'value': keyval[1], 'type': 'select', 'options': ['general use', 'ask to main user']}})


        elif keyval[0].lower() == 'Delete? (Y/N)':
            metadata['extra_fields'].update({keyval[0]: {'value': keyval[1], 'type': 'select', 'options': ['Y', 'N']}})

        else:
            metadata['extra_fields'].update({keyval[0]: {'value': keyval[1], 'type': field_type}})
    
        
    

    return json.dumps(metadata)

In [25]:
# The column "Technical Details" will get added to the body of the resource
def getBodyFromRow(row) -> str:
    for keyval in row.items():
        if keyval[0] == 'Technical Details':
            return f'<p>{keyval[1]}</p>'
    return ''

## Real stuff starts here

### Importing resources

Now we specify the file to be imported

In [17]:

# filename_example is the name of the file we want to read
filename_example = 'elab_resources_import_test_antibodies_2.csv'
filename_MB_test = 'Molecular_Bionics_Inventory_Jun_5_2024_test.csv'
filename_MB_merged = 'Molecular_Bionics_Inventory_Merged_2024-06-19.csv'



In [18]:
CSV_PATH = (parent_dir / relative_path / filename_MB_merged).resolve()

Quick check of a file

In [26]:
test_file = csv.DictReader(open(CSV_PATH, newline=''), delimiter=',', quotechar='"')
print(test_file.fieldnames)
print(test_file.line_num)

['Type', 'Delete? (Y/N)', 'Serial Number', 'Item Name *', 'Vendor', 'Catalog #', 'Owner', 'Location', 'Sub-location', 'Location Details', 'Price', 'Amount in Stock', 'Amount in Stock Units', 'Min to Stock', 'Max to Stock', 'Unit Size', 'URL', 'Technical Details', 'Expiration Date', 'Lot Number', 'CAS Number', 'Alternate Name', 'Antigen', 'Applications', 'Bottle Reference', 'Clonality', 'Clone', 'Concentration', 'Conjugation', 'Date Opened', 'Epitope', 'Formula', 'Isotype', 'Lifespan', 'Molecular Weight', 'Physical State', 'Primary vs Secondary', 'Purity', 'Raised in', 'Recognizes', 'Contact Person', 'Contact Phone', 'Date Installed', 'Date Purchased', 'Maintenance History', 'Serial #', 'University Tag #', 'Main user', 'User *', 'User to ask', 'Only for']
1


In [27]:
for row in test_file:
    test = body={'title': row['Item Name *'], 'body': getBodyFromRow(row), 'custom_id': row['Serial Number'], 'metadata': getMetadataFromRow(row)}
    print(test)

{'title': '(Created via API) Human ACE-2 PAb Alexa Fluor 594', 'body': '<p></p>', 'custom_id': 'B00001', 'metadata': '{"extra_fields": {"Type": {"value": "Antibodies (primary)", "type": "text"}, "Delete? (Y/N)": {"value": "N", "type": "text"}, "Vendor": {"value": "Bio-Techne R&D Systems, S.L.U.", "type": "text"}, "Catalog #": {"value": "FAB933T-100UG", "type": "text"}, "Owner": {"value": "Matilde Ghibaudi", "type": "text"}, "Location": {"value": "GBB_4 degrees", "type": "text"}, "Sub-location": {"value": "Reagents 1", "type": "text"}, "Location Details": {"value": "Blue box", "type": "text"}, "Price": {"value": "", "type": "text"}, "Amount in Stock": {"value": "", "type": "text"}, "Amount in Stock Units": {"value": "units", "type": "text"}, "Min to Stock": {"value": "", "type": "text"}, "Max to Stock": {"value": "", "type": "text"}, "Unit Size": {"value": "", "type": "text"}, "URL": {"value": "", "type": "text"}, "Expiration Date": {"value": "", "type": "text"}, "Lot Number": {"value":

### Import is done here: create items and modify them by filling in the fields

In [30]:
resource_dict = {'Antibodies (primary)': 15, 
                 'Antibodies (secondary)': 59,
                 'Fluorophores GBQ': 60,
                 'General Supply': 61,
                 'Reagents': 62,
                 'Equipment': 63,
                 'Solvents': 68}

# RESOURCE_CATEGORY_ID = 15 #IBEC Antibodies (Primary)
# RESOURCE_CATEGORY_ID = 59 #IBEC Antibodies (Secondary)
# RESOURCE_CATEGORY_ID = 60 #IBEC Fluorophores GBQ
# RESOURCE_CATEGORY_ID = 61 #IBEC General Supply
# RESOURCE_CATEGORY_ID = 62 #IBEC Reagents
# RESOURCE_CATEGORY_ID = 63 #IBEC Equipment
# RESOURCE_CATEGORY_ID = 68 #IBEC Solvents

In [ ]:
# Note: use encoding='utf-8-sig' in the open() call if your file has BOM (Byte Order Mark)
# Also make sure that the CSV file was saved as UTF-8 to avoid issues with special characters

with open(CSV_PATH, newline='') as csvfile: 
    
    # let's read the CSV using the standard "csv" library from python. No need for anything fancier.
    csvreader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
    
    # now we loop over each row in our CSV
    for row in csvreader:
        #we assign RESOURCE_CATEGORY_ID based on the 'Type' column taking the value from the distionary
        RESOURCE_CATEGORY_ID = resource_dict[row['Type']]
        
        # we can add a custom tag e.g. "-20°C freezer" to every row
        # the API allows setting tags during creation (POST) of a resource or experiment, so we use it here
        response = itemsApi.post_item_with_http_info(body={'category_id': RESOURCE_CATEGORY_ID, 
                                                           #'tags': ['-20°C freezer']
                                                           })
        locationHeaderInResponse = response[2].get('Location')
        print(f'The newly created item is here: {locationHeaderInResponse}')

        # that's our ID of the newly created resource
        itemId = int(locationHeaderInResponse.split('/').pop())
        print('The Item Id is: ' + str(itemId))

        # Patch the item to change its content:
        # the "Name" column becomes our title
        # the "Body" is generated from the "Comment" column content with the "getBodyFromRow()" function
        # for the "ID" column we match it to the "custom_id" property in elab
        # and the extra fields (metadata) is built with a function
        # the single line below will make all those changes at once
        itemsApi.patch_item(itemId, body={'title': row['Item Name *'], 'body': getBodyFromRow(row), 'custom_id': row['Serial Number'], 'metadata': getMetadataFromRow(row)})

### Modifying resources

What item you want to modify

In [41]:
itemId_to_modify = itemId
print(f'Item to modify: {itemId_to_modify}')

Item to modify: 48


Modify whatever needed

In [ ]:
itemsApi.patch_item(itemId, body={'title': 'Another title', 
                                  #'body': 'Main content text', 
                                  #'rating': 5
                                  })

## Importing experiments

In [ ]:
# Load the experiments api
experimentsApi = elabapi_python.ExperimentsApi(api_client)

# all experiments belonging to that user will get modified
target_userid = 2
# make sure this team group exists!
# to get its id go to /api/v2/teams/current/teamgroups
target_teamgroup = 2

# This is the permission setting we will assign to the experiments
# base:20 means "User + Admin" (see https://github.com/elabftw/elabftw/blob/b193d9fc738ab2635e07317ad83f8c5c1c50413a/src/enums/BasePermissions.php#L17)
canwrite = canread = { 'base': 20, 'teams': [], 'teamgroups': [target_teamgroup], 'users': [] }

# get a list of experiments for a given user
experiments = experimentsApi.read_experiments(owner=target_userid,limit=9999)
for exp in experiments:
    experimentsApi.patch_experiment(exp.id, body={'canread': json.dumps(canread), 'canwrite': json.dumps(canwrite)})